In [ ]:
"""
{% block stream %}
{:.output_stream}
```
{{ output.text }}
```
{% endblock stream %}
""";

In [ ]:
import nbformat, re
from nbconvert import MarkdownExporter
from traitlets.config import Config
from IPython.display import FileLink
from pathlib import Path
from pdb import set_trace

In [ ]:
def _exporter():
    exporter = MarkdownExporter(Config())
    exporter.exclude_input_prompt=True
    exporter.exclude_output_prompt=True
#     exporter.template_file = 'jekyll.tpl'
    exporter.template_path.append(str(Path.cwd().absolute()))
    return exporter

In [ ]:
_re_title = re.compile(r'^\s*#\s+([^\n]*)\n')
cell_type,outputs,source,code,text = 'cell_type','outputs','source','code','text'

def process_output(c,s,o):
    if c[cell_type]!=code or o is None: return s,o
    def _f(x):
        if text not in x: return x
        x[text] = re.sub(r'^(.*\S)',r'> \1',x[text], flags=re.MULTILINE)
        return x
    return s,[_f(o_) for o_ in o]

def process_title(c,s,o):
    if s.startswith('#hide'): return
    if c[cell_type] == code: return s,o
    if _re_title.search(s):
        s = '---\n' + _re_title.sub(r'title: "\1"', s) + '\n---'
        s = re.sub('^- ', '', s, flags=re.MULTILINE)
    return s,o

In [ ]:
def apply_all(x, fs, **kwargs):
    for f in fs:
        o = x.get(outputs,None)
        s,o = f(x, x[source], o, **kwargs) or (None,None)
        x[source]=s
        if s is None: x=None; break
        elif o is not None: x[outputs] = o
    return x

In [ ]:
def convert(fname, dest=None, cell_procs=None):
    fname = Path(fname)
    (fname.parent/'md_out').mkdir(exist_ok=True)
    if dest is None: dest = (fname.parent/'md_out'/fname.name).with_suffix('.md')
    if cell_procs is None: cell_procs = [process_title,process_output]
    with open(fname,'r') as f: nb = nbformat.reads(f.read(), as_version=4)
    nb['cells'] = [o for o in [apply_all(c, cell_procs) for c in nb['cells']] if o is not None]
    with open(dest,'w') as f: f.write(_exporter().from_notebook_node(nb)[0])

In [ ]:
name = 'delegation'
convert(Path.cwd()/f'{name}.ipynb')

In [ ]:
FileLink(f'{name}.md')

/home/jhoward/git/fastai_dev/dev/delegation.md